<a href="https://colab.research.google.com/github/Jaswanth-aditya/Conversational-AI-Toolkit-/blob/main/conversational_toolkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the OpenAI library, which is compatible with Groq API
!pip install --upgrade openai

# Import the required libraries
import os
import json
from openai import OpenAI
from google.colab import userdata

# Retrieve the Groq API key securely from Colab's user data
# To add your key, go to the key icon on the left panel (Secrets) and add a new secret named 'GROQ_API_KEY'
#
groq_api_key = userdata.get('GROQ_API_KEY')

# Initialize the Groq client with OpenAI SDK compatibility
client = OpenAI(
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1"
)

# Set the model to be used
# Groq provides several models, 'mixtral-8x7b-32768' is a good choice for this task
MODEL = "llama-3.3-70b-versatile"

print("Setup complete! Groq client initialized successfully.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 946.9/946.9 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.106.1
    Uninstalling openai-1.106.1:
      Successfully uninstalled openai-1.106.1
Setup complete! Groq client initialized successfully.


In [2]:
# Global state variables for conversation management
conversation_state = {
    "history": [],
    "turn_count": 0,
    "summarize_after_turns": 3,
    "turns_limit": 5
}

def add_message(role, content):
    """Adds a new message to the conversation history."""
    conversation_state["history"].append({"role": role, "content": content})
    conversation_state["turn_count"] += 1

def get_summary(history_to_summarize):
    """
    Generates a concise summary of the conversation history using Groq API.
    """
    prompt = "Summarize the following conversation concisely to preserve context for a chatbot. Maintain key details like names, topics, and important requests. Respond only with the summary."

    conversation_text = ""
    for msg in history_to_summarize:
        conversation_text += f"{msg['role']}: {msg['content']}\n"

    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": conversation_text}
            ],
            temperature=0.5
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"An error occurred during summarization: {e}")
        return "Summarization failed."

def manage_history():
    """
    Manages conversation history using both periodic summarization and truncation.
    """
    # Periodic summarization logic
    if conversation_state["turn_count"] > 0 and conversation_state["turn_count"] % conversation_state["summarize_after_turns"] == 0:
        print(f"*** Performing periodic summarization after turn {conversation_state['turn_count']} ***")
        summary = get_summary(conversation_state["history"])

        last_message = conversation_state["history"][-1]
        conversation_state["history"] = [
            {"role": "system", "content": f"The conversation so far has been summarized. Previous context: {summary}"},
            last_message
        ]
        conversation_state["turn_count"] = 1 # Reset turn count for the new "thread"

    # Truncation logic by number of turns
    if len(conversation_state["history"]) > conversation_state["turns_limit"]:
        print(f"*** Truncating history to the last {conversation_state['turns_limit']} turns ***")
        conversation_state["history"] = conversation_state["history"][-conversation_state["turns_limit"]:]

def get_completion(user_prompt):
    """
    Sends the current conversation history to Groq and gets a completion.
    """
    add_message("user", user_prompt)
    manage_history()

    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=conversation_state["history"],
            temperature=0.7,
            max_tokens=2048
        )
        assistant_response = response.choices[0].message.content.strip()
        add_message("assistant", assistant_response)
        return assistant_response
    except Exception as e:
        return f"An error occurred: {e}"

def display_history(title):
    """A helper function to display the current conversation state."""
    print(f"\n--- {title} ---")
    for msg in conversation_state["history"]:
        print(f"**{msg['role'].capitalize()}:** {msg['content']}")
    print(f"-----------------------------------\n")

In [3]:
# --- Demo 1: Truncation by number of turns ---
print("--- DEMO 1: Truncation by Number of Turns ---")
# Reset state for this demo
conversation_state = {"history": [], "turn_count": 0, "summarize_after_turns": 100, "turns_limit": 3}
sample_dialogue_1 = [
    "Hello, who are you?",
    "I am an AI assistant built by Groq. How can I help you?",
    "I'm looking for a new job. I have experience in software development and project management.",
    "Okay, I can help with that. What kind of roles are you interested in?",
    "I'd like to find senior developer or tech lead positions in the San Francisco Bay Area.",
    "Got it. I'll search for those roles for you.",
    "Thanks. Please also look for roles that involve Python and machine learning.",
    "Will do. Any other preferences?"
]

for i, prompt in enumerate(sample_dialogue_1):
    print(f"--- Turn {i+1} ---")
    response = get_completion(prompt)
    display_history(f"History after turn {i+1}")
    print(f"Assistant's Response: {response}")
    print("\n" + "="*50 + "\n")

# --- Demo 2: Periodic Summarization ---
print("--- DEMO 2: Periodic Summarization ---")
# Reset state for this demo
conversation_state = {"history": [], "turn_count": 0, "summarize_after_turns": 3, "turns_limit": 100}
sample_dialogue_2 = [
    "Hi, I want to book a flight to Paris.",
    "Sure, what are your travel dates?",
    "I want to fly out on December 20th and return on December 27th.",
    "Okay, searching for flights...",
    "Found some flights. How many people are traveling?",
    "Just one person. Also, can you check for hotels in the city center?"
]

for i, prompt in enumerate(sample_dialogue_2):
    print(f"--- Turn {i+1} ---")
    response = get_completion(prompt)
    display_history(f"History after turn {i+1}")
    print(f"Assistant's Response: {response}")
    print("\n" + "="*50 + "\n")

--- DEMO 1: Truncation by Number of Turns ---
--- Turn 1 ---

--- History after turn 1 ---
**User:** Hello, who are you?
**Assistant:** Hello. I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."
-----------------------------------

Assistant's Response: Hello. I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."


--- Turn 2 ---

--- History after turn 2 ---
**User:** Hello, who are you?
**Assistant:** Hello. I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."
**User:** I am an AI assistant built by Groq. How can I help you?
**Assistant:** Nice to meet you, Groq's AI assistant. I'm not in need of assistance at the moment, but it's great to know that you're here to help. I'm more of a conversational AI, so I'm happy to chat with you and learn more about your capabilities and what you can do. What kind of tasks or questions can you assist wi

In [4]:
# Define the JSON schema for information extraction
extraction_schema = {
    "name": "extract_contact_info",
    "description": "Extracts personal contact information like name, email, phone, location, and age from a user's chat message. This tool should only be used for information collection and not for general knowledge questions.",
    "parameters": {
        "type": "object",
        "properties": {
            "full_name": {
                "type": "string",
                "description": "The full name of the person.",
            },
            "email_address": {
                "type": "string",
                "description": "The email address of the person.",
            },
            "phone_number": {
                "type": "string",
                "description": "The phone number of the person, including the country code if available.",
            },
            "location": {
                "type": "string",
                "description": "The city and/or country of the person's location.",
            },
            "age": {
                "type": "integer",
                "description": "The age of the person.",
            }
        },
        "required": ["full_name", "email_address"]
    },
}

# The tools list that will be passed to the API
tools = [
    {
        "type": "function",
        "function": extraction_schema,
    }
]

def extract_info_with_groq(user_chat):
    """
    Sends a user chat to Groq to perform function calling for information extraction.
    """
    try:
        completion = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "user", "content": user_chat}
            ],
            tools=tools,
            tool_choice="auto",
            max_tokens=2048
        )

        tool_calls = completion.choices[0].message.tool_calls
        if tool_calls:
            for tool_call in tool_calls:
                if tool_call.function.name == "extract_contact_info":
                    extracted_data = json.loads(tool_call.function.arguments)
                    print("--- Structured Data Extracted ---")
                    print(json.dumps(extracted_data, indent=2))

                    # Validation of outputs against the schema
                    if "full_name" in extracted_data and "email_address" in extracted_data:
                        print("\nValidation Succeeded: 'full_name' and 'email_address' are present.")
                    else:
                        print("\nValidation Failed: Required fields are missing.")
                    return extracted_data
        else:
            print("No structured data was extracted. The model provided a regular message.")
            print(f"Model response: {completion.choices[0].message.content}")
            return None

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [5]:
# --- Sample 1: All information provided ---
print("--- Parsing Sample Chat 1 ---")
chat_1 = "Hi, my name is Alex Johnson. You can reach me at alex.johnson@example.com. My phone number is +1-555-123-4567, and I live in San Francisco. I'm 30 years old."
extract_info_with_groq(chat_1)
print("\n" + "="*50 + "\n")

# --- Sample 2: Partial information provided ---
print("--- Parsing Sample Chat 2 ---")
chat_2 = "Hello, my name is Emily Davis. My email is emilyd@test.co.uk. I'm 25."
extract_info_with_groq(chat_2)
print("\n" + "="*50 + "\n")

# --- Sample 3: Irrelevant chat, should not trigger the function ---
print("--- Parsing Sample Chat 3 ---")
chat_3 = "What is the capital of France?"
extract_info_with_groq(chat_3)
print("\n" + "="*50 + "\n")

--- Parsing Sample Chat 1 ---
--- Structured Data Extracted ---
{
  "age": 30,
  "email_address": "alex.johnson@example.com",
  "full_name": "Alex Johnson",
  "location": "San Francisco",
  "phone_number": "+1-555-123-4567"
}

Validation Succeeded: 'full_name' and 'email_address' are present.


--- Parsing Sample Chat 2 ---
--- Structured Data Extracted ---
{
  "age": 25,
  "email_address": "emilyd@test.co.uk",
  "full_name": "Emily Davis"
}

Validation Succeeded: 'full_name' and 'email_address' are present.


--- Parsing Sample Chat 3 ---
No structured data was extracted. The model provided a regular message.
Model response: The capital of France is Paris.


